# Experimenting with diy-search engine and elastic-search to build the context and generate responses from OpenAI

In [1]:
import json
import requests
import minsearch
from openai import OpenAI

In [2]:
with open('documents.json','rt') as f_in:
    docs = json.load(f_in)

In [3]:
documents = []

for course_dict in docs:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
index = minsearch.Index(text_fields=["question", "text", "section"],
                        keyword_fields=["course"]
)

index.fit(documents)

In [19]:
def search(q):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=q,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [31]:
def build_prompt(question, context):
    prompt = """You are a chatbot answering frequently asked questions for an online course. 
    Provide ANSWER to the QUESTION based on the CONTEXT given below. 
    Only state the facts from the CONTEXT else respond that information is not available.
    
    QUESTION = {question}
    CONTEXT = {context}
    ANSWER
    """
    prompt_output = prompt.format(question = question, context=context)
    return prompt_output

In [32]:
def rag_response(q):
    client = OpenAI()
    context = search(q)
    prompt = build_prompt(q, context)
    response = client.chat.completions.create(model='gpt-4o',
               messages = [{"role":"user","content":prompt}])
    return response.choices[0].message.content

In [34]:
query = "Do I need to know python to pass the course?"
rag_response(query)

'The provided context does not specify whether knowledge of Python is necessary to pass the course. Information is not available.'

In [35]:
search(query)

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the 

# Elastic Search
###Run Following docker container to host elasticsearch and forward ports 9200 and 9300 if not done automatically.

docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3

In [36]:
from elasticsearch import Elasticsearch

In [41]:
es_client = Elasticsearch('http://localhost:9200') 

In [42]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [47]:
for doc in documents:
    es_client.index(index=index_name,document=doc)

In [78]:
def elastic_search(query):
    search_query = {
                    "size": 5,
                    "query": {
                        "bool": {
                            "must": {
                                "multi_match": {
                                    "query": query,
                                    "fields": ["question^3", "text^2", "section"],
                                    "type": "best_fields"
                                }
                            },
                            "filter": {
                                "term": {
                                    "course": "data-engineering-zoomcamp"
                                }
                            }
                        }
                    }
                }
    response = es_client.search(index=index_name,body=search_query)
    result = []
    for hit in response['hits']['hits']:
        result.append(hit['_source'])
    return result

In [79]:
def rag_elastic(q):
    client = OpenAI()
    context = elastic_search(q)
    prompt = build_prompt(q, context)
    response = client.chat.completions.create(model='gpt-4o',
               messages = [{"role":"user","content":prompt}])
    return response.choices[0].message.content

In [86]:
query = "python version required for course ?"

In [87]:
rag_elastic(query)

'The recommended Python version for the course is Python 3.9 for simplicity and stability, but Python 3.10 and 3.11 should also work fine.'